In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon, gamma = individual
    if C <= 0 or epsilon <= 0 or gamma <= 0:
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(1, 10), (0.001, 1), (0.001, 5)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1]),
                  lambda: random.uniform(*parameter_ranges[2])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 100
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 0.1
mutation_rate = 0.9
num_generations = 75

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon, gamma = best_individual

# Test the best SVR model
best_svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"Gamma: {gamma}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [4.1302468739565255, 0.0, 3.364022268016906], MAPE: inf
Individu: [6.003348157150277, 0.3410876578047368, 0.160087863871728], MAPE: 24.473684719656045
Individu: [0.0, 0.8420939563109505, 3.6530894547960595], MAPE: inf
Individu: [9.38586368604902, 0.12687703186034577, 2.485483444728615], MAPE: 24.3738697342323
Individu: [0.0, 0.3447322920481881, 2.6619144718120404], MAPE: inf
Individu: [2.198764070823864, 0.9891200936196163, 2.7190714429378313], MAPE: 20.390900629871226
Individu: [6.180747103003793, 0.81684569648959, 1.5797239563631975], MAPE: 20.390900629871226
Individu: [1.3865107387751512, 0.6387178229681882, 0.0], MAPE: inf
Individu: [9.157045222982536, 0.031144276434082983, 1.5047930974887338], MAPE: 29.404566425910435
Individu: [5.197470443215139, 0.7304307967490222, 0.2630898881382271], MAPE: 20.390900629871226
Individu: [3.256228403528549, 0.4488873003811286, 2.039804997289641], MAPE: 20.36567911353774
Individu: [8.300147328073589, 0.3668235302789717, 0.305

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 3  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 36  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[19.1901229  17.96865089 18.6432249  16.71724341 17.04750642 18.6432249
 19.1901229  17.79403712 17.96865089 23.56919214 18.32585554 16.71724341]
